<a href="https://colab.research.google.com/github/ZellW/API_Development/blob/master/Style_Replicator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

ANTHROPIC_API_KEY = 'ANTHROPIC_API_KEY' # Replace with your Anthropic API key

def replicate_writing_style(sample_texts, prompt, feedback_history):
    system_prompt = "You are an AI agent that replicates the writing style, tone, and voice demonstrated in the provided sample texts. Focus on matching the style, tone, and voice, regardless of the content or topic. Take into account the feedback history to improve your replication over time."

    messages = [
        {"role": "user", "content": f"Sample Texts:\n{sample_texts}\n\nFeedback History:\n{feedback_history}\n\nPrompt:\n{prompt}"},
    ]

    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }

    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 500,
        "temperature": 0.7,
        "system": system_prompt,
        "messages": messages,
    }

    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    generated_text = response.json()['content'][0]['text']

    return generated_text

def evaluate_style_similarity(sample_texts, generated_text):
    system_prompt = "You are an AI evaluator that assesses the similarity in writing style, tone, and voice between the generated text and the provided sample texts. Provide a similarity score between 0 and 1, where 0 means no similarity and 1 means a perfect match in style, tone, and voice, regardless of the content or topic. Respond with only the numeric score."

    messages = [
        {"role": "user", "content": f"Sample Texts:\n{sample_texts}\n\nGenerated Text:\n{generated_text}\n\nPlease provide a similarity score between 0 and 1, where 0 means no similarity and 1 means a perfect match in writing style, tone, and voice, regardless of the content or topic. Respond with only the numeric score."},
    ]

    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }

    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 50,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }

    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text'].strip()

    try:
        similarity_score = float(response_text)
    except ValueError:
        print("The AI evaluator did not provide a valid similarity score. Setting the score to 0.")
        similarity_score = 0.0

    return similarity_score

####
sample_texts = []
feedback_history = ""
style_similarity_threshold = 0.8

while True:
    user_input = input("Enter a sample of your writing style (or 'done' to finish): ")

    if user_input.lower() == 'done':
        break

    sample_texts.append(user_input)


while True:
    prompt = input("Enter a prompt to generate text in your writing style (or 'quit' to exit): ")

    if prompt.lower() == 'quit':
        break

    # Generate text in the user's writing style
    generated_text = replicate_writing_style(sample_texts, prompt, feedback_history)
    print("\nGenerated Text:")
    print(generated_text)

    # Evaluate style similarity
    style_similarity_score = evaluate_style_similarity(sample_texts, generated_text)
    print(f"\nStyle Similarity Score: {style_similarity_score}")

    if style_similarity_score >= style_similarity_threshold:
        print("The generated text closely matches your writing style, tone, and voice!")
    else:
        # Get user feedback
        feedback = input("Provide feedback to improve the style, tone, and voice of the generated text: ")
        feedback_history += f"Prompt: {prompt}\nGenerated Text: {generated_text}\nFeedback: {feedback}\n\n"

print("Thank you for using the AI Writing Style Replicator!")